# Brain Age Prediction Challenge Tutorial

# Load libraries that will be used for the challenge

In [14]:
import mne
from mne.channels import find_layout
from mne.time_frequency import psd_array_welch
from natsort import natsorted
import numpy as np
import os
import pandas as pd
from pyprep.find_noisy_channels import NoisyChannels
import scipy.io as sio
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
import wget

## Step 1: Obtaining Data
We will use two datasets for this challenge. One is the dataset provided with the Codalab challenge, the other is the openly available dataset of Healthy Brain. Both datasets consist of eyes-closed (EC) and eyes-open (EO) resting-state EEG recordings using a system with 129 channels. Additionally, both datasets cover a similar age-range, which makes the Healthy Brain data an optimal addition. Therefore, we will first load the Healthy Brain dataset.
The fastest way to get (just) the resting-state EEG data from the Healthy Brain dataset is to download a program like Cyberduck and connect to the Amazon Web Services S3 bucket (see [here](http://fcon_1000.projects.nitrc.org/indi/s3/index.html) on how to connect). Once we are connected to the server, we can use the following script to only download the EEG recordings. Alternatively, one may download all dataset chunks of the full dataset [here](http://fcon_1000.projects.nitrc.org/indi/cmi_healthy_brain_network/sharing_neuro.html).

In [15]:
#get_participants = wget.download("https://fcp-indi.s3.amazonaws.com/data/Projects/HBN/EEG/participants.tsv") #get the participant list
participants = pd.read_csv('participants.tsv',sep='\t')[["participant_id", "Age"]] #load tsv file and select subject and target column
participants.rename(columns={"participant_id":"id", "Age":"age"}, inplace = True)
participants = participants.loc[~participants.id.duplicated()][["id", "age"]] #exclude duplicates
participants = participants.loc[~participants.age.isnull()][["id", "age"]] #exclude subjects without age information
participants.to_csv("H:/Healthy_Brain_Network/participants.csv") #save the csv for later steps (change the path to your desire)
print(len(participants.id))

3553


We have 3553 participants in the Healthy Brain dataset, however, not all of them will contain (usable) EEG data as we will see later. The next step is to download the EEG data from the server. Therefore, we specify a directory to save the data to (for me that is "H:/Healthy_Brain_Network/EEG_raw_tutorial/") and check whether we already downloaded some of the files, in case you interrupt the script and want to run it again at a later point. Other than that we download the file, open it (unfortunately it is in a matlab file), and save it.
Note: by now Healthy Brain also seems to offer BIDS format, but at the time I first downloaded it, it was not available and so the code is based on the matfiles.

In [ ]:
for subject in tqdm(participants.id):
    if os.path.exists("H:/Healthy_Brain_Network/EEG_raw_tutorial/{}.npy".format(subject)) == False:
        url = 'https://fcp-indi.s3.amazonaws.com/data/Projects/HBN/EEG/{}/EEG/raw/mat_format/RestingState.mat'.format(subject)
        try:
            file = wget.download(url)
            sub_dict = sio.loadmat(file)
            filename = "H:/Healthy_Brain_Network/EEG_raw_tutorial/{}.npy".format(subject)
            np.save(filename, sub_dict)
        except:
            continue

  1%|          | 44/3569 [01:45<13:45,  4.27it/s]   

Now, that the Healthy Brain data is downloaded, get the data from the challenge, and proceed with pre-processing.

## Step 2: Preprocessing

1. Download the EEG channel location file from the Healthy Brain [website](http://fcon_1000.projects.nitrc.org/indi/cmi_healthy_brain_network/sharing_neuro.html). We need to load it into the following code.
2. We will create a directory for the preprocessed files such that for each subject in the directory (in my case: "H:/Healthy_Brain_Network/EEG_preprocessed_tutorial/"), we will get an EC and EO folder with the respective recordings.
3. The recordings of each subject contain up to 5 EC (40sec each) and 5 EO (20sec each) measurements in sequence. Therefore, we use the event timings to split them and save them separately such that they are similar to the competition data)
4. The preprocessing consists of:
    - checking for bad channels (exclude recording if more than 30 bad channels)
    - interpolation of bad channels
    - filtering (0.1-45Hz)
    - downsampling (100Hz)

Note: usually, you would first try to detect bad channels, interpolate, filter and downsample the full recordings before splitting them into epochs. However, in the spirit of making the Healthy Brain data as similar as possible to the competition data (only has one 20sec EO and 40sec EC recording for each participant), the preprocessing is done epoch-wise.
Note: %%capture is used to not print out all the output that the pyprep library spits out (which may make your IDE slow)

### Healthy Brain Data Preprocessing

In [ ]:
%%capture

#Create info for MNE raw object
montage = mne.channels.read_custom_montage("H:/Healthy_Brain_Network/GSN_HydroCel_129.sfp") #load EEG channel location file
info = mne.create_info(montage.ch_names,500,'eeg')
info.set_montage(montage)

#Settings for downsampling and filtering
l_freq = 0.1
h_freq = 45
sfreq = 100

#Get EC and EO raw EEGs and separate them into two folders per subject
for sub in tqdm(os.listdir("H:/Healthy_Brain_Network/EEG_raw_tutorial")):
    sub = sub.replace(".npy", "")
    save_path = "H:/Healthy_Brain_Network/EEG_preprocessed_tutorial/{}".format(sub) #set your desired output path here!

    if not os.path.exists(save_path): #check if already preprocessed, else create directories
        os.mkdir(save_path)
        os.mkdir("{}/EC".format(save_path))
        os.mkdir("{}/EO".format(save_path))
    else:
        continue

    #Load file and check if it contains data for all channels
    file = np.reshape(np.load("H:/Healthy_Brain_Network/EEG_raw_tutorial/{}.npy".format(sub), allow_pickle = True),1)
    try:
        data = file[0]["EEG"][0]["data"][0] #EEG data (indexing is a bit ugly because it is a matfile)
        if data.shape[0] < 129: #less than 129 channels?
            continue
    except: #no data
        continue

    #Get events
    events = pd.DataFrame(file[0]["EEG"][0]["event"][0][0]) #event times to get
    EO_times = events[events.type == "20  "]["sample"].array.astype(int)
    EC_times = events[events.type == "30  "]["sample"].array.astype(int)

    #Split recording into according EC and EO trials
    n = 1
    for i,j in zip(EO_times,EC_times):
        try:
            EO = data[:,int(i):int(j)]
            EC = data[:,int(j):int(EO_times[n])]
        except IndexError:
            continue

        #MNE for interpolation of bad channels, filtering, & downsampling
        for k, trial in enumerate([EO, EC]):
            raw = mne.io.RawArray(trial, info) #create mne raw object
            try:
                #Use pyprep library to detect noisy channels
                nc = NoisyChannels(raw)
                nc.find_all_bads()
                if len(nc.get_bads()) > 30: #if more than ~1/4 of channels are bad
                    continue
            except:
                continue

            raw.info['bads'].extend(nc.get_bads())
            raw.interpolate_bads(reset_bads=True) #interpolation
            raw.filter(l_freq,h_freq) #filtering
            raw.resample(sfreq) #downsampling
            raw_data, _ = raw[:]

            if k == 0:
                np.save("{}/EO/{}.npy".format(save_path, n), raw_data)
            else:
                np.save("{}/EC/{}.npy".format(save_path, n), raw_data)

        n += 1

Additional Note: Initially, I tried to pick the 64 EEG channels that overall have the least occurences of being bad for all subjects. The idea was to reduce the number of parameters. However, using a subset of channels or trying dimensionality reduction (PCA) did lead to slightly worse predictions in the end. Therefore, I suggest working with all channels for best possible performance, unless you want to reduce the cost of training (which may make sense for hyperparameter search).

In the next cell, we will turn the preprocessed data into a Power-Spectral Density (PSD) representation that yields the power for each frequency from 1-45Hz for each channel. This step could have been included in the previous cell, however, I probed around with the preprocessed data and creating PSDs was only one option that eventually gave the best result.

The input into the later model will be a combination of EC and EO recording, therefore, we will check if every EC recording has a corresponding EO recording (they may have been filtered out e.g. due to bad channels in the previous step). Then, we use Welch's method to obtain the PSDs and save them in the output path ("H:/Healthy_Brain_Network/psds_tutorial" in my case).

Note: one approach that I tested was to use the FOOOF library (https://fooof-tools.github.io/fooof/) to reduce noise by selecting oscillatory patterns over non-periodic parts of the signals. During evaluation, this step did not seem to make a difference, so I ended up not using it. I left the code for it in the script below as I think it is generally a good additional filter step to implement (feel free to out-comment it).

In [ ]:
# from fooof import FOOOF

data_path = "H:/Healthy_Brain_Network/EEG_preprocessed_tutorial" #directory of preprocessed Healthy Brain data
for sub in os.listdir(data_path):
    save_path = "H:/Healthy_Brain_Network/psds_tutorial/{}".format(sub) #set your desired output path!

    if not os.path.exists(save_path):
        os.mkdir(save_path)
        os.mkdir("{}/EC".format(save_path))
        os.mkdir("{}/EO".format(save_path))
    else:
        continue

    _, EC, EO = os.walk("{}/{}".format(data_path, sub))
    available = [file for file in EC[2] if file in EO[2]] #check if file for EC and EO exists

    for file in available:
        for trial in ["EC", "EO"]:
            data = np.load("{}/{}/{}/{}".format(data_path, sub, trial, file))
            raw = mne.io.RawArray(data, info)
            raw_data, _ = raw[:]
            psds,freqs = psd_array_welch(raw_data, sfreq, fmin=1, fmax=45, n_fft=sfreq)

            #outcommented code was used for fooof
            # psds_clean = np.zeros_like(psds)
            # for chan in range(psds.shape[0]):
            #         fm = FOOOF(verbose = False)
            #         fm.fit(freqs, psds[chan])
            #         clean_chan = 10**fm.fooofed_spectrum_-10**fm._ap_fit
            #         psds_clean[chan] = clean_chan

            np.save("{}/{}/{}".format(save_path, trial, file), psds) #psds_clean for FOOOF-cleaned data

### Brain Age Challenge Data Preprocessing

Download Brain Age Challenge Data and unpack everything in a directory (for me: "H:/NeuroTex/raw_train_tutorial/"). The preprocessing resembles the one for the Healthy Brain data, however, this time we don't want to drop recordings with >30 bad channels. The same directory structure is applied, however, for the challenge we only have one recording for EC and EO each per participant.

Note: the preprocessing of the validation/test data is not shown because it follows exactly the same procedure. Only the input and output paths need to be changed.

In [ ]:
%%capture
sfreq = 100
l_freq = 0.1
h_freq = 45

participants = pd.read_csv("H:/NeuroTex/train_subjects.csv") #load the csv files of the participants
for sub in participants.id.values:
    save_path = "H:/NeuroTex/EEG_preprocessed_tutorial/{}".format(sub) #set save path!

    if not os.path.exists(save_path):
        os.mkdir(save_path)
        os.mkdir("{}/EC".format(save_path))
        os.mkdir("{}/EO".format(save_path))
    else:
        continue

    #Might need to pad with zeros as subject 1 is "0001", etc.
    pad = 4-len(str(sub))
    id = "0"*pad+str(sub)
    for trial in ["EO", "EC"]:
        fname = "H:/NeuroTex/raw_train_tutorial/subj{}_{}_raw.fif".format(id, trial)
        raw = mne.io.read_raw_fif(fname, preload=True)

        try:
            nc = NoisyChannels(raw)
            nc.find_all_bads()
        except:
            continue

        raw.info['bads'].extend(nc.get_bads())
        raw.interpolate_bads(reset_bads=True)
        raw.filter(l_freq,h_freq)
        raw.resample(sfreq)
        raw_data, _ = raw[:]

        np.save("{}/{}/1.npy".format(save_path, trial), raw_data)

...do the same for validation/test data.

As for the Healthy Brain dataset, we now create the PSDs for the challenge data (again, do the same for validaition/test!).

In [ ]:
# from fooof import FOOOF

sfreq = 100
#load a raw object to get the channel names (does not have to be done for each subject repeatedly)
orig = mne.io.read_raw_fif("H:/NeuroTex/raw_validation/subj1201_EO_raw.fif", preload=True)
ch_names = orig.ch_names
info = mne.create_info(ch_names,sfreq,'eeg')

data_path = "H:/NeuroTex"
for sub in natsorted(os.listdir("{}/EEG_preprocessed_tutorial".format(data_path))):
    save_path = "{}/psds_tutorial/{}".format(data_path, sub)
    if not os.path.exists(save_path):
        os.mkdir(save_path)
        os.mkdir("{}/EC".format(save_path))
        os.mkdir("{}/EO".format(save_path))

    else:
        continue

    for trial in ["EC", "EO"]:
        try:
            data = np.load("{}/EEG_preprocessed_tutorial/{}/{}/1.npy".format(data_path, sub, trial))
        except:
            continue

        raw = mne.io.RawArray(data, info)
        raw_data, _ = raw[:]
        psds,freqs = psd_array_welch(raw_data, sfreq, fmin=1, fmax=45, n_fft=sfreq)

        #outcommented code was used for fooof
        # psds_clean = np.zeros_like(psds)
        # for chan in range(psds.shape[0]):
        #         fm = FOOOF(verbose = False)
        #         fm.fit(freqs, psds[chan])
        #         clean_chan = 10**fm.fooofed_spectrum_-10**fm._ap_fit
        #         psds_clean[chan] = clean_chan

        np.save("{}/{}/1.npy".format(save_path, trial), psds) #psds_clean for FOOOF-cleaned psds

## Step 3: Load Data

In this step, we will load the PSD files, scale them (using sklearn), and combine the respective EC (129x45) and EO (129x45) recordings to get the samples that we will put into the model in the next step. The dimensions of a sample will be 129x90, which we need to flatten as they are passed to a Support Vector Machine later. Meanwhile, we also store the age (target) for each recording in Y_train...

In [ ]:
#Load data, scale, and flatten (sklearn expects 2d input [sample x values], we need to flatten all 2D samples)
scaler = StandardScaler()
train = "H:/Healthy_Brain_Network/psds_tutorial"

df_hb = pd.read_csv("H:/Healthy_Brain_Network/participants.csv") #load the previously saved participant csv

X_train_hb = []
Y_train_hb = []
for sub in os.listdir(train):
    if len(os.listdir("{}/{}/EC".format(train, sub))) > 0: #check if subject contains recorded (that have not been filtered out); we could have checked EO just as well
        for sample in os.listdir("{}/{}/EC".format(train, sub)):
            data_EC = np.load("{}/{}/EC/{}".format(train, sub, sample))
            data_EC = scaler.fit_transform(data_EC).flatten()
            data_EO = np.load("{}/{}/EO/{}".format(train, sub, sample))
            data_EO = scaler.fit_transform(data_EO).flatten()
            data = np.hstack((data_EC, data_EO))
            X_train_hb.append(data)
            Y_train_hb.append(df_hb.loc[df_hb.id == sub, "age"].iloc[0])

X_train_hb = np.array(X_train_hb) #scaled psds [n_samples x 129*90]
Y_train_hb = np.array(Y_train_hb) #ages

Now, load the challenge data. Again, the same processing steps are taken as for the Healthy Brain data.

In [ ]:
#Load data, scale, and flatten (sklearn expects 2d input [sample x values], we need to flatten all 2D samples)
scaler = StandardScaler()
path = "H:/NeuroTex/"
train = "H:/NeuroTex/psds_tutorial"
df_nt = pd.read_csv("H:/NeuroTex/train_subjects.csv") #load the csv file downloaded from the competition website

X_train_nt = []
Y_train_nt = []
for sub in natsorted(os.listdir(train)):
    try:
        data_EC = np.load("{}/{}/EC/1.npy".format(train, sub))
        data_EC = scaler.fit_transform(data_EC).flatten()
        data_EO = np.load("{}/{}/EO/1.npy".format(train, sub))
        data_EO = scaler.fit_transform(data_EO).flatten()
        data = np.hstack((data_EC, data_EO))
        X_train_nt.append(data)
        Y_train_nt.append(df_nt.loc[df_nt.id == int(sub), "age"].iloc[0])
    except FileNotFoundError:
        continue

X_train_nt = np.array(X_train_nt) #scaled psds [n_samples x 129*90]
Y_train_nt = np.array(Y_train_nt) #ages

Combine Healthy Brain and the competition data:

In [ ]:
X_train = np.concatenate((X_train_nt, X_train_hb))
Y_train = np.hstack((Y_train_nt, Y_train_hb))

## Step 4: Model

### Hyperparameter Tuning

To detect the optimal hyperparameters, I will use nested cross-validation in combination with grid-search. Essentially, this approach combines the standard cross-validation with an inner loop that looks for the best hyperparameter combination and then evaluates on the outer loop. For an overview, see [here](https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/), from where this code is adapted.
For the grid search, I used the log-scale for parameters C and gamma, which is deemed the standard approach (e.g.: H. Frohlich and A. Zell, "Efficient parameter selection for support vector machines in classification and regression via model-based global optimization," Proceedings. 2005 IEEE International Joint Conference on Neural Networks, 2005., 2005, pp. 1431-1436 vol. 3, doi: 10.1109/IJCNN.2005.1556085. The paper also proposes an alternative approach, in case you are interested.)

Essentially, the code below will try out every parameter combination in the inner folds, then train the best combination on 9/10 outter folds and evaluate on the left-out fold. It will then print the MAE and the optimal parameter combination for that run.
Note: nested cross-validation is extremely expensive, so I suggest you iteratively *try out small subsets of the below suggested hyperparameters*. You may also decrease the number of inner folds to increase the speed.

In [ ]:
outer_results = list()
#Outer CV
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
for train_ix, val_ix in cv_outer.split(X_train):
    #split data
    x_train, x_val = X_train[train_ix, :], X_train[val_ix, :]
    y_train, y_val = Y_train[train_ix], Y_train[val_ix]

    #Inner CV
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    model = SVR()
    #define search space
    param_grid = dict()
    param_grid["C"] = np.logspace(-18, 9, num=20, base=2).tolist() #best: 0.0009765625
    param_grid["kernel"] = ["linear", "rbf", "sigmoid", "poly"] #best: linear
    param_grid["gamma"] = np.logspace(-28, 9, num=20, base=2).tolist() #best: 3.725290298461914e-09 (however not used for linear kernel)
    search = GridSearchCV(model, param_grid, scoring='neg_mean_absolute_error', cv=cv_inner, refit=True, error_score='raise')
    result = search.fit(x_train, y_train) #runs fit with all sets of parameters
    best_model = result.best_estimator_ #get the best performing model fit on the whole training set
    yhat = best_model.predict(x_val) #evaluate model on the hold out dataset
    mae_svm = mean_absolute_error(y_val, yhat) #evaluate the model
    outer_results.append(mae_svm) #store result
    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae_svm, result.best_score_, result.best_params_))

#summarize estimated performance of the model
outer_results = np.array(outer_results)
print('Mean Absolute Error: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

After several runs, I figured that a good hyperparameter combination is given by a linear Kernel combined with a C of 0.0009765625. This combination was then used to make the predictions. I also tried out different models like a random forest (I won't put it in here to not overload the notebook, but I can provide it in case you would like to have it).

### Run Model

Fit the model on the training data with the desired parameters. This may take a while because we have over 14.000 samples with a dimension of 129x90 each.

In [ ]:
model = SVR(kernel="linear", gamma = 3.725290298461914e-09, C=0.0009765625)
model.fit(X_train, Y_train)

Make predictions.
Note: run the same preprocessing and loading script for the validation/test, make the X_test object for the validation/test data as shown for the training set, and put it into the model to make the predictions.

In [ ]:
Y_pred = model.predict(X_test)

Lastly, create the csv file that will be compressed and uploaded to the challenge.

In [ ]:
pd.DataFrame(zip(np.arange(2001, 2401), Y_pred), columns=['id', 'age']).to_csv(os.path.join('submissions', f'output_path'),index=False) #for the test set